# Case Study: Age in American Politics

In the last few section workbooks, we've reviewed Python basics, used a little `pandas`, wrangled and cleaned data, and carried out an exploratory data analysis where we generated a number of basic visualizations. This week, we'll extend this a bit futher, incorporating all that we've learned so far AND considering what we discussed in lecture last week. 

Your goal is to, without much structure provided, consider what we discussed in lecture last week and utilize the data provided to answer the question(s) we discussed in class last week: 

1. Does Congress have an age problem?
2. Is this problem exclusive to one of the two major parties?

The data you have to start with are available here: [congress-terms.csv](https://github.com/fivethirtyeight/data/tree/master/congress-age). They were used in [this piece](https://fivethirtyeight.com/features/both-republicans-and-democrats-have-an-age-problem/) at FiveThirtyEight. Note, there is an entry for every member of congress who has served at any point during a particular congress between January 1947 and Februrary 2014. One thing to keep in mind is the fact that elections have occurred since 2014 that are not included in this dataset. Getting up-to-date data will be explored in Part III of this notebook.


# Part I: Setup & Data

In this section you'll want to:
- **import any packages** you'll need for your analysis
    - Reminder: we'll be doing web scraping, so you'll need to include the following:
        - `import requests`
        - `import bs4` 
        - `from bs4 import BeautifulSoup`

- **get the data**:
    - read the Congress dataset in (URL: https://raw.githubusercontent.com/fivethirtyeight/data/master/congress-age/congress-terms.csv)
    - Read, understand, and run the webscraping code below

In [4]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style

import datetime

import requests
import bs4
from bs4 import BeautifulSoup

import json
from pandas.io.json import json_normalize

In [5]:
# READ CONGRESS DATA IN HERE

In [7]:
## CODE TO SCRAPE TABLE FROM WIKIPEDIA
## UNDERSTAND AND RUN ALL FOLLOWING CELLS

# specify webpage we want to scrape 
wiki = 'https://en.wikipedia.org/wiki/Demography_of_the_United_States'
req = requests.get(wiki)
soup = BeautifulSoup(req.content) # get contents of web page

In [9]:
wikitables = soup.findAll("table", 'wikitable') # get tables
# extract the tables we want
tbl1 = wikitables[5] 
tbl2 = wikitables[6]
tbl1

<table class="wikitable" style="text-align:right">
<tbody><tr>
<th>Years</th>
<th>1820</th>
<th>1830</th>
<th>1840</th>
<th>1850</th>
<th>1860</th>
<th>1870</th>
<th>1880</th>
<th>1890</th>
<th>1900
</th></tr>
<tr>
<td align="left">Median age of the total population</td>
<td>16.7</td>
<td>17.2</td>
<td>17.8</td>
<td>18.9</td>
<td>19.4</td>
<td>20.2</td>
<td>20.9</td>
<td>22.0</td>
<td>22.9
</td></tr>
<tr>
<td align="left">Median age of males</td>
<td>16.6</td>
<td>17.2</td>
<td>17.9</td>
<td>19.2</td>
<td>19.8</td>
<td>20.2</td>
<td>21.2</td>
<td>22.3</td>
<td>23.3
</td></tr>
<tr>
<td align="left">Median age of females</td>
<td>16.8</td>
<td>17.3</td>
<td>17.8</td>
<td>18.6</td>
<td>19.1</td>
<td>20.1</td>
<td>20.7</td>
<td>21.6</td>
<td>22.4
</td></tr></tbody></table>

In [ ]:
# create some empty dataframes
# note the tables aren't the same size. ugh.
new_tbl1 = pd.DataFrame(columns=range(0,10), index = range(0,3)) # I know the size 
new_tbl2 = pd.DataFrame(columns=range(0,13), index = range(0,3))

In [ ]:
# get the column names for our first table
ind=0
cols_list = []
for header in tbl1.find_all('tr'): # specify HTML tags
    header_name = header.find_all('th') # tag containing column names
    for head in header_name:
        cols_list.append(head.get_text()) # get the text from between the tags
new_tbl1.columns = [s.replace('\n','') for s in cols_list] # get rid of new line characters in column names

In [ ]:
# fill in the contents for our first table
row_marker = -1
for row in tbl1.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td') # different tag than above for table contents
    for column in columns:
        new_tbl1.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

In [ ]:
# get the column names for the second table
ind=0
cols_list = []
for header in tbl2.find_all('tr'):
    header_name = header.find_all('th')
    for head in header_name:
        cols_list.append(head.get_text())
new_tbl2.columns = [s.replace('\n','') for s in cols_list] 

In [ ]:
# fill in contents for second table
row_marker = -1
for row in tbl2.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_tbl2.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

In [ ]:
# of course there are new line characters to get rid of 
new_tbl2 = new_tbl2.replace(r'[\\n,\n]',' ', regex=True) 
new_tbl1 = new_tbl1.replace(r'[\\n,\n]',' ', regex=True) 

In [ ]:
# set Years as index
new_tbl1.set_index(['Years'], inplace=True)
new_tbl2.set_index(['Years'], inplace=True)

In [ ]:
# get it into tidy data format
age = new_tbl1.join(new_tbl2) # join two tables into one by index
age = age.transpose() # flip it so observations in rows
age = age.reset_index() # add year back in as a column
age.columns = ['year', 'age', 'median_males', 'median_females'] # rename columns
age = age.apply(pd.to_numeric, errors='coerce') # fix type of information stored
age # flipping finally

# Part II: Descriptive & Exploratory Analyses

Once you understand your dataset, you'll want to:
- **understand** (describe) what's going on in the data
    - how many observations?
    - what variables do you have? what variable types are here
    - which will you need to answer the question?
- Carry out **EDA**
    - understand the relationships and trends for the variables in your dataset
    - generate exploratory visualizations
    - answer the exploratory questions we posed in lecture

## Descriptive Analysis

In [ ]:
# DESCRIPTIVE ANALYSIS OF CONGRESS DATASET HERE

In [ ]:
# DESCRIPTIVE ANALYSIS OF AGE DATASET HERE

## Exploratory Analysis

In [ ]:
# YOUR CODE HERE

# Part III: More data?

Once you have a good handle on what's going on in the dataset, how can you get the data you need to include recent elections? I'll note that in the FiveThirtyEight piece where these data came from, they mention using the NYT's Congress API. However, this has moved to Propublica since publication of that piece: https://projects.propublica.org/api-docs/congress-api/.

This will require you to read API documentation and understand it. Additionally, you will likely need additional packages. It may be helpful to consider the following packages: `json`, `requests` & `pandas.io.json`

Note that this is optional and will take time to figure it out. Accessing data from an API is not required here, but if you're interested, have at it!

In [ ]:
# YOUR CODE HERE